# Introduction

Below various steps of data processing are implemented. 
The starting point is the excel outputs of the tool that has already been developed in Matlab.

### Importing Required Libraries

In [5]:
import numpy as np, pandas as pd
from os import listdir
from os.path import join
import os
import matplotlib, datetime
import matplotlib.pyplot as plt

plt.style.use('ggplot')
%matplotlib inline

from ipywidgets import widgets, interact, interact_manual, interactive, Layout
import ipywidgets as widgets
import xlrd
import fileupload

# from tkinter import *
# from tkinter.filedialog import askopenfilename

### Inputs File Selection

_Reminder: Pls select the output from the .m executable. Note: Check for open windows!!_

In [10]:
filename = r'D:\Work\PLC\PLC Data\Playing\Original output_2019-06-24.xlsx'

In [11]:
# # Get file name
# Tk().withdraw()
# filename = askopenfilename()

# # Read file
# Df = pd.read_excel(filename)

In [ ]:
Df = pd.read_excel(filename)

In [ ]:
df = Df.copy()

# Data Processing

### Remove Un-used Columns

Please select string contained in columns to be removed.

In [ ]:
text = widgets.Text()

@interact
def select_text(String='write text, e.g. off'):
    text.value = str(String)

In [ ]:
# Drop columns including string
s = text.value
df = df.loc[:,~df.columns.str.contains(s, case=False)]

In [ ]:
# Drop columns including "." --> Additional Time Columns
df = df.loc[:,~df.columns.str.contains('.', regex=False)]

In [ ]:
# Format time & drop un-used time columns
df['Time'] = df['Time *10^6'] * 10**(-6)
def read_date(date):
    return xlrd.xldate.xldate_as_datetime(date, 0)
df['Time'] = pd.to_datetime(df['Time'].apply(read_date), errors='coerce')

df['datetime'] = pd.to_datetime(df['Time'])
df = df.set_index('datetime')
df.drop(['Time *10^6'], axis=1, inplace=True)

df['day'], df['time'] = df.Time.dt.date, df.Time.dt.time
df.drop(['Time'], axis=1, inplace=True)

In [ ]:
df.time = df.time.apply(lambda x: x.replace(microsecond=0))

In [ ]:
D = df.copy()

### Remove Data from Other Days

Print available dates & ask user to select one.

In [ ]:
date = widgets.RadioButtons(
            options=list(D['day'].unique()),
            description='Select date:')
display(date)

In [ ]:
# Drop raws including other dates
d = date.value
D = D[D['day'] == d]

In [ ]:
ddd = D.copy()

### Clean Outliers

In [ ]:
### Clean Outliers
#
def ident_outliers(D, c):
    m = D[c].mean(); s = D[c].std()
    return np.abs(D[c] - m) / s > 3

def clean_outliers(D, c):
    D.iloc[ident_outliers(D, c), c] = np.nan
    return D[c]

In [ ]:
for c in ddd.columns:
    try:
        clean_outliers(ddd, c)
    except:
        pass

### Add Air Flow & Inverter Data

Air Flow Data.

Scecify number of entries / changes in LPM.

In [ ]:
e = widgets.IntText(description='Entries:')
display(e)

Specify starting LPM.

In [ ]:
starting_lpm = widgets.BoundedFloatText(value=750.0, min=0, max=1000.0, step=10.0,
                                      description='Starting LPM:')
display(starting_lpm)

In [ ]:
dc = {}
def f(t, v, n):
    if n:
        dc[t] = v

for i in range(e.value):

    t = widgets.Text(value='00:00:00', description='Time:')
    v = widgets.FloatText(value=starting_lpm.value, description='LPM:')
    n = widgets.ToggleButton(value=False, description='Add')
    UI = widgets.HBox([t, v, n])

    out = widgets.interactive_output(f, {'t': t, 'v': v, 'n': n})

    display(UI, out)

In [ ]:
ddd['aLPM'] = starting_lpm.value

def f(t, v):
    D = date.value; T = datetime.datetime.strptime(t, '%H:%M:%S').time()
    y, mo, d, h, mi, s = D.year, D.month, D.day, T.hour, T.minute, T.second
    
    ddd.loc["%s-%s-%s %s:%s:%s"%(y, mo, d, h, mi, s):, "aLPM"] = v

for k, v in dc.items():
    f(k, v)

Inverter Data.

Scecify number of entries / changes in inverter.

In [ ]:
ei = widgets.IntText(description='Entries:')
display(ei)

Specify starting value.

In [ ]:
starting_hz = widgets.BoundedFloatText(value=37.5, min=0, max=50.0, step=.5,
                                      description='Starting Hz:')
display(starting_hz)

In [ ]:
dci = {}
def f(t, v, n):
    if n:
        dci[t] = v

for i in range(ei.value):

    t = widgets.Text(value='00:00:00', description='Time:')
    v = widgets.FloatText(value=starting_hz.value, description='Hz:')
    n = widgets.ToggleButton(value=False, description='Add')
    UI = widgets.HBox([t, v, n])

    out = widgets.interactive_output(f, {'t': t, 'v': v, 'n': n})

    display(UI, out)

In [ ]:
ddd['iHz'] = starting_hz.value

def f(t, v):
    D = date.value; T = datetime.datetime.strptime(t, '%H:%M:%S').time()
    y, mo, d, h, mi, s = D.year, D.month, D.day, T.hour, T.minute, T.second
    
    ddd.loc["%s-%s-%s %s:%s:%s"%(y, mo, d, h, mi, s):, "iHz"] = v

for k, v in dci.items():
    f(k, v)

In [ ]:
ddd.head()

# Plotting & Visualizing

In [ ]:
c = widgets.SelectMultiple(options=list(ddd.columns), value=[list(ddd.columns)[0], list(ddd.columns)[1]],
        rows=6, description='Columns:', layout=Layout(width='80%'))
t = widgets.SelectionRangeSlider(options=ddd['time'], index=(0, len(ddd['time'])-1), description='X-Axis:', 
        layout=Layout(width='80%', height='80px'))

UI = widgets.VBox([c, t])

In [ ]:
def f(cols, t):
    ts = t[0]; te = t[1]
    ax = ddd.plot(x='time', y=list(cols), style=".", figsize=(14,6))
    ax.set_xlim(ts, te)
    ax.plot()
    
out = widgets.interactive_output(f, {'cols': c, 't': t})

display(UI, out)

# Save File

Specify directory.

In [ ]:
dir_name = filedialog.askdirectory()

Specify file name.

In [ ]:
default_v = '%s-%s-%s out.xlsx'%(date.value.year, date.value.month, date.value.day)
f_name = widgets.Text(value=default_v, description='File Name:')
display(f_name)

In [ ]:
button = widgets.Button(description="Save")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        ddd.to_excel(join(dir_name, f_name.value))
        print("File saved.")

button.on_click(on_button_clicked)